In [1]:
## tensorflowエキスパートクイックスタート
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from IPython.core.debugger import Pdb

# Pdb().set_trace()
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

# データセットをシャッフルし、バッチ化するために tf.data を使います。
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

# Pdb().set_trace()


# Kerasの model subclassing API を使ってtf.kerasモデルを作ります。
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# モデルのインスタンスを作成
model = MyModel()



In [2]:
# 自作関数による学習の実行
# ロス関数，オプティマイザを設定
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()


# 学習結果を記録するためのメトリクスを取得
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


# tf.GradientTapeを使ってモデルを訓練します。
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)


@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)


EPOCHS = 5

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    #     Pdb().set_trace()
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))
  
  # 次のエポック用にメトリクスをリセット
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()





If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 0.12754115462303162, Accuracy: 96.15666961669922, Test Loss: 0.059603236615657806, Test Accuracy: 97.97999572753906
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "/Users/takumi/.venvs/TFPractice-Y7NqnZAF/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-5c2b0c8b4aab>", line 4

TypeError: object of type 'NoneType' has no len()

In [ ]:
#### メトリクスをテスト
m = tf.keras.metrics.Accuracy()

m.update_state([[1], [2], [3], [4]], [[0], [2], [3], [1]])
# m([[1], [2], [3], [4]], [[0], [2], [3], [4]]) # これも同じ動き
m.result().numpy()


In [48]:
### tf.dataの練習
import tensorflow as tf
import random
import pathlib
import matplotlib.pyplot as plt

AUTOTUNE = tf.data.experimental.AUTOTUNE

data_root_orig = tf.keras.utils.get_file(origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
                                         fname='flower_photos', untar=True)
data_root = pathlib.Path(data_root_orig)

# 
attributions = (data_root/"LICENSE.txt").open(encoding='utf-8').readlines()[4:]
attributions = [line.split(' CC-BY') for line in attributions]
attributions = dict(attributions)

# 画像のパスリストの取得
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)

# 画像の総数
image_count = len(all_image_paths)

label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
# ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

label_to_index = dict((name, index) for index,name in enumerate(label_names))
# {'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}

all_image_labels = [label_to_index[pathlib.Path(path).parent.name] for path in all_image_paths]
# First 10 labels indices:  [2, 3, 4, 2, 1, 3, 2, 0, 3, 2]

########################################################################################################
img_path = all_image_paths[0]
img_raw = tf.io.read_file(img_path)# 生のバイナリデータ
img_tensor = tf.image.decode_image(img_raw) # numpy配列に変換
# Pdb().set_trace()
########################################################################################################


def caption_image(image_path):
  image_rel = pathlib.Path(image_path).relative_to(data_root)
  return "Image (CC BY 2.0) " + ' - '.join(attributions[str(image_rel)].split(' - ')[:-1])


def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=3) # numpy配列に変換
  image = tf.image.resize(image, [192, 192])      # リサイズ
  image /= 255.0                                  # 正規化
  return image


def load_and_preprocess_image(path):
  image = tf.io.read_file(path)                 # 生のバイナリデータ
#   Pdb().set_trace()
  return preprocess_image(image)


# # パス文字列によるデータセット
path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)

# # 画像データセット
image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)

# # ラベルデータセット
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))

# #(image, label) というペアのデータセット
image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))
print(image_label_ds)

ds = image_label_ds.shuffle(buffer_size=image_count)
print(ds)
ds = ds.repeat()
print(ds)
ds = ds.prefetch(buffer_size=AUTOTUNE)
print(ds)
BATCH_SIZE = 32
ds = ds.batch(BATCH_SIZE)
print(ds)

# for n,image in enumerate(image_ds.take(4)):
#     print(image.shape)

# import matplotlib.pyplot as plt

# plt.figure(figsize=(8,8))
# for n,image in enumerate(image_ds.take(4)):
#   plt.subplot(2,2,n+1)
#   plt.imshow(image)
#   plt.grid(False)
#   plt.xticks([])
#   plt.yticks([])
#   plt.xlabel(caption_image(all_image_paths[n]))
#   plt.show()

<ZipDataset shapes: ((192, 192, 3), ()), types: (tf.float32, tf.int64)>
<ShuffleDataset shapes: ((192, 192, 3), ()), types: (tf.float32, tf.int64)>
<RepeatDataset shapes: ((192, 192, 3), ()), types: (tf.float32, tf.int64)>
<PrefetchDataset shapes: ((192, 192, 3), ()), types: (tf.float32, tf.int64)>
<BatchDataset shapes: ((None, 192, 192, 3), (None,)), types: (tf.float32, tf.int64)>


In [39]:
image_ds.take(3)

<TakeDataset shapes: (192, 192, 3), types: tf.float32>

In [49]:
# tf.data.Datasetのハンズオン（非公式） https://qiita.com/Suguru_Toyohara/items/820b0dad955ecd91c7f3
# 非公式仮説　https://qiita.com/studio_haneya/items/1138e427367e93cd2ab8

import tensorflow as tf
import tensorflow.keras as keras

dataset = tf.data.Dataset.from_tensor_slices(tf.range(10))

for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


In [ ]:
# tf recordの利用
[x_train, y_train], [x_test, y_test] = tf.keras.datasets.mnist.load_data()

def make_example(image, label):
    return tf.train.Example(features=tf.train.Features(feature={
        'x' : tf.train.Feature(float_list=tf.train.FloatList(value=image)),
        'y' : tf.train.Feature(int64_list=tf.train.Int64List(value=label))
    }))

def write_tfrecord(images, labels, filename):
    writer = tf.io.TFRecordWriter(filename)
    for image, label in zip(images, labels):
        ex = make_example(image.ravel().tolist(), [int(label)])
        writer.write(ex.SerializeToString())
    writer.close()

write_tfrecord(x_train, y_train, '../mnist_train.tfrecord')
write_tfrecord(x_test, y_test, '../mnist_test.tfrecord')

In [54]:
import numpy as np
np.arange(25)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24])

In [59]:
import traceback
import contextlib

# Some helper code to demonstrate the kinds of errors you might encounter.
@contextlib.contextmanager
def assert_raises(error_class):
  try:
    yield
  except error_class as e:
    print('Caught expected exception \n  {}:'.format(error_class))
    traceback.print_exc(limit=2)
  except Exception as e:
    raise e
  else:
    raise Exception('Expected {} to be raised but no error was raised!'.format(
        error_class))

'''
@tf.function
def f(x):
  v = tf.Variable(1.0)
  v.assign_add(x)
  return v


with assert_raises(ValueError):
    f(1.0)
'''
# 複数回同じVariableを作成する処理を行ってはいけない．∴はじめに関数が呼び出されたときにのみvariableを作成することができる．
# 以下のように，元の一回のみ変数定義を行うように書くことで，エラーを回避できる．
class Count(tf.Module):
  def __init__(self):
    self.count = None
  
  @tf.function
  def __call__(self):
    if self.count is None:
      self.count = tf.Variable(0)
    return self.count.assign_add(1)

c = Count()
print(c())
print(c())



tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
